In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
exchange_info = pd.read_csv("../data/exchange/exchangeInfo.csv", index_col=[0])
exchange_info.head()

,symbol,status,baseAsset,baseAssetPrecision,quoteAsset,quotePrecision,quoteAssetPrecision,baseCommissionPrecision,quoteCommissionPrecision,orderTypes,icebergAllowed,ocoAllowed,quoteOrderQtyMarketAllowed,isSpotTradingAllowed,isMarginTradingAllowed,filters,permissions
0,ETHBTC,TRADING,ETH,8,BTC,8,8,8,8,"['LIMIT', 'LIMIT_MAKER', 'MARKET', 'STOP_LOSS_...",True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...","['SPOT', 'MARGIN']"
1,LTCBTC,TRADING,LTC,8,BTC,8,8,8,8,"['LIMIT', 'LIMIT_MAKER', 'MARKET', 'STOP_LOSS_...",True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...","['SPOT', 'MARGIN']"
2,BNBBTC,TRADING,BNB,8,BTC,8,8,8,8,"['LIMIT', 'LIMIT_MAKER', 'MARKET', 'STOP_LOSS_...",True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...","['SPOT', 'MARGIN']"
3,NEOBTC,TRADING,NEO,8,BTC,8,8,8,8,"['LIMIT', 'LIMIT_MAKER', 'MARKET', 'STOP_LOSS_...",True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...","['SPOT', 'MARGIN']"
4,QTUMETH,TRADING,QTUM,8,ETH,8,8,8,8,"['LIMIT', 'LIMIT_MAKER', 'MARKET', 'STOP_LOSS_...",True,True,True,True,False,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...",['SPOT']


In [3]:
available_assets = np.unique(np.append(exchange_info.baseAsset.values, exchange_info.quoteAsset.values))
len(available_assets)

459

In [4]:
json_data = {"nodes": [{"id": x, "group": 1} for x in available_assets],
            "links": [{"source": row["quoteAsset"], "target": row["baseAsset"], "value": 1} for i, row in exchange_info.iterrows()]}


In [5]:
with open('../crypto_market_network/crypto_market.json', 'w') as f:
    json.dump(json_data, f)

In [6]:
crypto_graph = {k: set() for k in available_assets}
for i, row in exchange_info.iterrows():
    crypto_graph[row['quoteAsset']].add(row['baseAsset'])

In [10]:
arbitrage_paths = {}
def find_arbitrage_paths(graph, start, current_path=[]):
    global arbitrage_paths
    if start not in arbitrage_paths:
        arbitrage_paths[start] = []
    
    current_node = graph[start] if len(current_path)==0 else graph[current_path[-1]]
    if len(current_node)==0:
        return None
    if start in current_node:
        arbitrage_paths[start] += [current_path + [start]]
        return None
    for future_node in current_node:
        if future_node not in current_path:
            find_arbitrage_paths(graph, start, current_path + [future_node])
    return None 

In [14]:
for coin in available_assets:
    find_arbitrage_paths(crypto_graph, coin)
    arbitrage_paths[coin] = sorted(arbitrage_paths[coin], key=len)

In [17]:
for coin in available_assets:
    arbitrage_paths[coin] = sorted(arbitrage_paths[coin], key=len)

In [18]:
with open('../data/exchange/arbitrage_paths.json', 'w') as f:
    json.dump(arbitrage_paths, f)

In [19]:
arbitrage_paths['BTC']

[['TUSD', 'BTC'],
 ['PAX', 'BTC'],
 ['DAI', 'BTC'],
 ['TUSD', 'BTC'],
 ['PAX', 'BTC'],
 ['DAI', 'BTC'],
 ['ETH', 'PAX', 'BTC'],
 ['ETH', 'TUSD', 'BTC'],
 ['BNB', 'PAX', 'BTC'],
 ['BNB', 'DAI', 'BTC'],
 ['BNB', 'TUSD', 'BTC'],
 ['BNB', 'USDC', 'BTC'],
 ['ETH', 'PAX', 'BTC'],
 ['ETH', 'TUSD', 'BTC'],
 ['BNB', 'PAX', 'BTC'],
 ['BNB', 'DAI', 'BTC'],
 ['BNB', 'TUSD', 'BTC'],
 ['BNB', 'USDC', 'BTC'],
 ['ETH', 'BNB', 'PAX', 'BTC'],
 ['ETH', 'BNB', 'DAI', 'BTC'],
 ['ETH', 'BNB', 'TUSD', 'BTC'],
 ['ETH', 'BNB', 'USDC', 'BTC'],
 ['ETH', 'BNB', 'PAX', 'BTC'],
 ['ETH', 'BNB', 'DAI', 'BTC'],
 ['ETH', 'BNB', 'TUSD', 'BTC'],
 ['ETH', 'BNB', 'USDC', 'BTC']]